# Steered MD analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import nglview as ng
from IPython.display import display
import pytraj as pt

In [ ]:
import yaml
with open('../.defaults/config', 'r') as file:
    _settings = yaml.load(file, Loader=yaml.FullLoader)

## Load data

Select data:

In [ ]:
system = widgets.Dropdown(options=os.listdir('../systems'), description='System:')
display(system)
state = widgets.RadioButtons(options=os.listdir(f'../systems/{system.value}'), description='State:')
display(state)
folder = widgets.Text(description='Folder:')
display(folder)

Go to folder:

In [ ]:
if folder.value == '':
    folder_name = 'seeded-md'
elif 'seeded-md' in folder.value:
    folder_name = folder.value
else:
    folder_name = f'seeded-md_{folder.value}'
os.chdir(f'../systems/{system.value}/{state.value}/{folder_name}/steering')

Load PLUMED output:

In [ ]:
data = pd.read_csv('steering.dat', delim_whitespace=True)
columns = data.columns[2:]
data = data.iloc[:,:-2]
data.columns = columns
data.index = data['time']/1000

## Plot data

This section is for plotting the CV outputs to check whether the sMD simulation was successful. It is recommended to set up any additional analysis here and run this notebook to check each sMD trajectory.

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))
ax.plot(data.index, data.iloc[:,1], alpha=0.7, label='CV')

ax.legend()
ax.set_xlim(data.index[0], data.index[-1])
ax.set_xlabel('time/ns', size=14)
ax.set_ylabel('CV value/units', size=14)

## Sample trajectory

Select snapshot number and set steering duration:

In [ ]:
snapshot_number = widgets.IntText(value=100)
traj_duration = widgets.IntText(value=data.index.max()//10*10)
display(widgets.HBox([widgets.Label('Snapshot number:'), snapshot_number]))
display(widgets.HBox([widgets.Label('Trajectory duration/ns:'), traj_duration]))

In [ ]:
last_frame = int(traj_duration.value/(data.index[2]-data.index[1]))
snapshot_idxs = np.linspace(0, last_frame, snapshot_number.value, dtype=int)

Check that the snapshots sample the conformational space well:

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))
ax.plot(data.iloc[snapshot_idxs].index, data.iloc[:,1].iloc[snapshot_idxs]*10, alpha=0.7, label='Hinge')

Visualize trajectory:

In [ ]:
reference = pt.load('../../../../../inputs/reference.pdb')

In [ ]:
trajectory = pt.load('steering_dry.nc', top='../../system-setup/system_dry.prm7')
trajectory = pt.align(trajectory, ':450-800&(@CA,C,N,O)', ref=reference, ref_mask=':450-800&(@CA,C,N,O)')

In [ ]:
view = ng.show_pytraj(trajectory)
view.clear_representations()
view.add_representation('cartoon', color='magenta')
view.add_component(reference)
view.component_1.clear_representations()
view.component_1.add_representation('cartoon', color='lightgreen')
view.display()

Save snapshots:

In [ ]:
if not os.path.exists('../snapshots'):
    os.mkdir('../snapshots')

In [ ]:
frame_idxs = []
for i, idx in enumerate(snapshot_idxs):
    snapshot = pt.load('steering.nc', top='../../system-setup/system.prm7', frame_indices=[idx])
    if not os.path.exists(f'../snapshots/snapshot_{i+1}.rst7'):
        pt.io.save_traj(f'../snapshots/snapshot_{i+1}.rst7', snapshot, format='rst7')
        os.rename(f'../snapshots/snapshot_{i+1}.rst7.1', f'../snapshots/snapshot_{i+1}.rst7')
    frame_idxs.append(f'{i+1} {idx}\n')

In [ ]:
with open('../snapshots/snapshot_idxs.txt', 'w') as file:
    file.writelines(frame_idxs)